In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"]="false"

In [2]:
# ! uv pip uninstall --system 'tensorflow'
! uv pip install --system --no-index --find-links='/kaggle/input/latest-mdc-whls/whls' 'pymupdf' # 'vllm' 'triton' 'logits-processor-zoo' 'numpy<2'

Using Python 3.11.13 environment at: /usr
Resolved 1 package in 68ms
Prepared 1 package in 503ms
Installed 1 package in 15ms
 + pymupdf==1.26.1


In [3]:
# ! uv pip install /kaggle/input/mdcfitz/pymupdf-1.26.3-cp39-abi3-manylinux_2_28_x86_64.whl
! uv pip install vllm --no-index --find-links file:///kaggle/input/mdcllm
! uv pip install logits-processor-zoo==0.1.10 --no-index --find-links file:///kaggle/input/mdcllm
! uv pip install triton==3.2.0 --no-index --find-links file:///kaggle/input/mdcllm

Using Python 3.11.13 environment at: /usr
Resolved 145 packages in 461ms
Prepared 54 packages in 41.40s
Uninstalled 23 packages in 2.35s
Installed 54 packages in 405ms
 + airportsdata==20250706
 + astor==0.8.1
 + blake3==1.0.5
 + compressed-tensors==0.10.2
 + depyf==0.18.0
 + diskcache==5.6.3
 + fastapi-cli==0.0.8
 + fastapi-cloud-cli==0.1.1
 + gguf==0.17.1
 + httptools==0.6.4
 - httpx==0.28.1
 + httpx==0.27.2
 + interegular==0.3.3
 + lark==1.2.2
 + llguidance==0.7.30
 - llvmlite==0.43.0
 + llvmlite==0.44.0
 + lm-format-enforcer==0.10.11
 + mistral-common==1.6.3
 + msgspec==0.19.0
 - numba==0.60.0
 + numba==0.61.2
 - nvidia-cublas-cu12==12.5.3.2
 + nvidia-cublas-cu12==12.6.4.1
 - nvidia-cuda-cupti-cu12==12.5.82
 + nvidia-cuda-cupti-cu12==12.6.80
 - nvidia-cuda-nvrtc-cu12==12.5.82
 + nvidia-cuda-nvrtc-cu12==12.6.77
 - nvidia-cuda-runtime-cu12==12.5.82
 + nvidia-cuda-runtime-cu12==12.6.77
 - nvidia-cudnn-cu12==9.3.0.75
 + nvidia-cudnn-cu12==9.5.1.17
 - nvidia-cufft-cu12==11.2.3.61
 + nvi

In [4]:
! mkdir -p /tmp/src

In [5]:
%%writefile /tmp/src/helpers.py
import logging, os, kagglehub, inspect
from pathlib import Path
import polars as pl

os.environ["TOKENIZERS_PARALLELISM"]="False"
IS_KAGGLE_ENV = sum(['KAGGLE' in k for k in os.environ]) > 0
os.environ["KAGGLE_IS_COMPETITION_RERUN"] = "1"

IS_KAGGLE_SUBMISSION = bool(os.getenv("KAGGLE_IS_COMPETITION_RERUN"))

COMP_DIR = Path(('/kaggle/input/make-data-count-finding-data-references' if IS_KAGGLE_SUBMISSION else kagglehub.competition_download('make-data-count-finding-data-references')))
PDF_DIR = COMP_DIR / ('test' if IS_KAGGLE_SUBMISSION else 'train') / 'PDF'
WORKING_DIR = Path(('/kaggle/working/' if IS_KAGGLE_ENV else '.working/'))

DOI_LINK = 'https://doi.org/'

DEFAULT_LOG_LEVEL = os.getenv("LOG_LEVEL", "DEBUG").upper() if not IS_KAGGLE_SUBMISSION else "WARNING"
LOG_FILE_PATH = os.getenv("LOG_FILE", "logs/project.log")
LOG_DIR = Path(LOG_FILE_PATH).parent

LOG_DIR.mkdir(parents=True, exist_ok=True)

LOG_FORMAT = "%(levelname)s %(asctime)s  [%(filename)s:%(lineno)d - %(funcName)s()] %(message)s"
LOG_DATEFMT = "%Y-%m-%d %H:%M:%S"

def get_logger(name=None):
    if name is None:
        frame = inspect.currentframe()
        if frame is None or frame.f_back is None:
            name = "__main__"
        else:
            name = frame.f_back.f_globals.get("__name__", "__main__")

    logger = logging.getLogger(name)

    if not logger.handlers:
        logger.setLevel(DEFAULT_LOG_LEVEL)
        formatter = logging.Formatter(fmt=LOG_FORMAT, datefmt=LOG_DATEFMT)
        ch = logging.StreamHandler()
        ch.setLevel(DEFAULT_LOG_LEVEL)
        ch.setFormatter(formatter)
        fh = logging.FileHandler(LOG_FILE_PATH)
        fh.setLevel(DEFAULT_LOG_LEVEL)
        fh.setFormatter(formatter)
        logger.addHandler(ch)
        logger.addHandler(fh)
        logger.propagate = False
    return logger

def is_doi_link(name: str) -> pl.Expr:
    return pl.col(name).str.starts_with(DOI_LINK)

def string_normalization(name: str) -> pl.Expr:
    return pl.col(name).str.normalize("NFKC").str.replace_all(r"[^\p{Ascii}]", '').str.replace_all(r"https?://zenodo\.org/record/(\d+)", r" 10.5281/zenodo.$1 ")

def get_df(parse_dir: str):
    records = []
    txt_files = list(Path(parse_dir).glob('*.txt'))
    for txt_file in txt_files:
        id_ = txt_file.stem
        with open(txt_file, 'r') as f:
            text = f.read()
        records.append({'article_id': id_, 'text': text})
    return pl.DataFrame(records).with_columns(string_normalization('text').alias('text'))
## remove
def assume_type_bak(df: pl.DataFrame) -> pl.DataFrame:
    return (
        df.with_columns(pl.when(is_doi_link('dataset_id').or_(pl.col('dataset_id').str.starts_with('SAMN'))).then(pl.lit('Primary')).otherwise(pl.lit('Secondary')).alias('type'))
    )

def assume_type(df: pl.DataFrame) -> pl.DataFrame:
    """Better rule-based classification using context keywords"""
    
    def classify_type(window, dataset_id):
        if pd.isna(window):
            window = ""
        window_lower = window.lower()
        
        # Strong Primary indicators
        if any(phrase in window_lower for phrase in [
            "we deposited", "we uploaded", "we submitted", "deposited by us",
            "our dataset", "we generated", "we collected", "we sequenced"
        ]):
            return "Primary"
        
        # Strong Secondary indicators  
        if any(phrase in window_lower for phrase in [
            "et al", "previously published", "downloaded from", 
            "obtained from", "retrieved from", "described by"
        ]):
            return "Secondary"
        
        # Fallback to identifier-based logic
        if is_doi_link_simple(dataset_id) or dataset_id.startswith('SAMN'):
            return "Primary"
        else:
            return "Secondary"
    
    types = [classify_type(w, d) for w, d in df.select(['window', 'dataset_id']).rows()]
    return df.with_columns(pl.Series('type', types))

def is_doi_link_simple(dataset_id):
    return dataset_id.startswith('https://doi.org/')
# wait
def assume_type_improved(df: pl.DataFrame) -> pl.DataFrame:
    """Use context windows for Primary/Secondary classification"""
    
    # Extract features from context windows
    primary_patterns = [
        r"(?i)\b(we\s+(collected|generated|created|obtained|sequenced|measured))",
        r"(?i)\b(our\s+dataset|newly\s+generated|custom\s+dataset)",
        r"(?i)\b(experimental\s+data|original\s+data|novel\s+dataset)",
        r"(?i)\b(deposited|uploaded|submitted|archived)\b.*\b(by\s+us|in\s+this\s+study)"
    ]
    
    secondary_patterns = [
        r"(?i)\b(previously\s+published|existing\s+dataset|publicly\s+available)",
        r"(?i)\b(obtained\s+from|downloaded\s+from|retrieved\s+from)",
        r"(?i)\b(reanalyz|secondary\s+analysis|meta-analysis)",
        r"(?i)\b(Smith\s+et\s+al|Jones\s+et\s+al|\w+\s+et\s+al\.?\s+\(20\d{2}\))",  # Citations
        r"(?i)\b(reported\s+by|described\s+by|published\s+by)"
    ]
    
    # Score based on context
    primary_scores = []
    secondary_scores = []
    
    for window in df['window']:
        if pd.isna(window):
            window = ""
        
        primary_score = sum(1 for pattern in primary_patterns 
                           if re.search(pattern, window))
        secondary_score = sum(1 for pattern in secondary_patterns 
                             if re.search(pattern, window))
        
        primary_scores.append(primary_score)
        secondary_scores.append(secondary_score)
    
    return df.with_columns([
        pl.Series('primary_score', primary_scores),
        pl.Series('secondary_score', secondary_scores)
    ]).with_columns(
        pl.when(pl.col('primary_score') > pl.col('secondary_score'))
          .then(pl.lit('Primary'))
          .when(pl.col('secondary_score') > pl.col('primary_score'))
          .then(pl.lit('Secondary'))
          .otherwise(
              # Fallback to your current logic when context is unclear
              pl.when(is_doi_link('dataset_id') | pl.col('dataset_id').str.starts_with('SAMN'))
                .then(pl.lit('Primary'))
                .otherwise(pl.lit('Secondary'))
          )
          .alias('type')
    )

def score(df, gt, on, tag='all'):
    hits = gt.join(df, on=on)
    tp = hits.height
    fp = df.height - tp
    fn = gt.height - tp
    f1 = 2 * tp / (2 * tp + fp + fn) if (2 * tp + fp + fn) != 0 else 0.0
    return f"{tag} - f1: {f1:.4f} [{tp}/{fp}/{fn}]"

def evaluate(df, on=['article_id', 'dataset_id']):
    gt = pl.read_csv(COMP_DIR/'train_labels.csv').filter(pl.col('type')!='Missing')
    return (
        score(df, gt, on),
        score(df.filter(is_doi_link('dataset_id')), gt.filter(is_doi_link('dataset_id')), on, 'doi'),
        score(df.filter(~is_doi_link('dataset_id')), gt.filter(~is_doi_link('dataset_id')), on, 'acc'),
    )

Writing /tmp/src/helpers.py


In [6]:
%%writefile /tmp/src/parse.py
import argparse
from pathlib import Path
import pymupdf
from helpers import get_logger, PDF_DIR

l = get_logger()

def pdf_to_txt(output_dir: Path):
    output_dir.mkdir(parents=True, exist_ok=True)
    pdf_files = list(PDF_DIR.glob("*.pdf")) + list(PDF_DIR.glob("*.PDF"))
    existing_txt_files = {f.stem for f in output_dir.glob("*.txt")}
    for pdf_file in pdf_files:
        txt_file = output_dir / f"{pdf_file.stem}.txt"
        if pdf_file.stem in existing_txt_files:
            continue
        try:
            text = ""
            with pymupdf.open(pdf_file) as doc:
                for page in doc:
                    text += page.get_text()
            txt_file.write_text(text, encoding='utf-8')
        except Exception:
            pass

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('output_dir', type=Path, help='Directory to save text files')
    args = parser.parse_args()
    pdf_to_txt(args.output_dir)

if __name__ == "__main__":
    main()

Writing /tmp/src/parse.py


In [7]:
%%writefile /tmp/src/check_parse.py
import polars as pl
from pathlib import Path
from helpers import *

l=get_logger()

def gt_dataset_id_normalization(name:str) -> pl.Expr:
    return (
        pl.when(is_doi_link(name))
        .then(pl.col(name).str.split(DOI_LINK).list.last())
        .otherwise(name)
        .str.to_lowercase()
    )

def main():
    if IS_KAGGLE_SUBMISSION:
        l.debug('skipping check_parse for submission')
        return
    df = (
        get_df('/tmp/train_parse')
        .with_columns(pl.col('text').str.replace_all('\s+', '').str.to_lowercase().alias('text'))
    )

    gt = (
        pl.read_csv(COMP_DIR/'train_labels.csv')
        .filter(pl.col('article_id').is_in(df['article_id']))
        .filter(pl.col('type')!='Missing')
        .with_columns(gt_dataset_id_normalization('dataset_id').alias('norm_id'))
    )

    l.info(f"pymupdf misses: {gt.join(df, on='article_id').with_columns(hit=pl.col('text').str.contains(pl.col('norm_id'), literal=True)).filter(~pl.col('hit')).height} dataset_ids")

if __name__=='__main__': main()

Writing /tmp/src/check_parse.py


In [8]:
%%writefile /tmp/src/getid.py
import re
import polars as pl
from typing import Optional, Tuple

from helpers import *

COMPILED_PATTERNS = {
    'ref_header_patterns': [re.compile(r'\b(R\s*E\s*F\s*E\s*R\s*E\s*N\s*C\s*E\s*S|BIBLIOGRAPHY|LITERATURE CITED|WORKS CITED|CITED WORKS|ACKNOWLEDGEMENTS)\b[:\s]*', re.IGNORECASE)],
    'citation_pattern': re.compile(r'^\s*(\[\d+\]|\(\d+\)|\d+\.|\d+\)|\d+(?=\s|$))\s*'),
    'first_citation_patterns': [
        re.compile(r'^\s*\[1\]\s*'),
        re.compile(r'^\s*\(1\)\s*'),
        re.compile(r'^\s*1\.\s*'),
        re.compile(r'^\s*1\)\s*'),
        re.compile(r'^\s*1(?=\s|$)'),
    ],
}

l = get_logger()

def find_last_reference_header(text: str, header_patterns: list[re.Pattern]) -> Optional[int]:
    last_match_idx = None
    for pattern in header_patterns:
        matches = list(pattern.finditer(text))
        if matches:
            last_match_idx = matches[-1].start()
    return last_match_idx

def find_last_first_citation(text: str) -> Optional[int]:
    lines = text.splitlines()
    last_match_line = None
    for line_num, line in enumerate(lines):
        line = line.strip()
        for pattern in COMPILED_PATTERNS['first_citation_patterns']:
            if pattern.match(line):
                next_lines = lines[line_num:line_num+3]
                if any(COMPILED_PATTERNS['citation_pattern'].match(l.strip()) for l in next_lines[1:]):
                    last_match_line = line_num
                break
    return last_match_line

def find_reference_start(text: str) -> Optional[int]:
    lines = text.splitlines()
    last_first_citation = find_last_first_citation(text)
    if last_first_citation is not None:
        return last_first_citation
    start_search_idx = int(len(lines) * 0.5)
    for i in range(start_search_idx, len(lines)):
        line = lines[i].strip()
        if COMPILED_PATTERNS['citation_pattern'].match(line):
            next_lines = lines[i:i+3]
            if sum(1 for l in next_lines if COMPILED_PATTERNS['citation_pattern'].match(l.strip())) >= 2:
                for j in range(i, max(-1, i-10), -1):
                    if not COMPILED_PATTERNS['citation_pattern'].match(lines[j].strip()):
                        return j + 1
                return max(0, i-10)
    return None

def split_text_and_references(text: str) -> Tuple[str, str]:
    header_idx = find_last_reference_header(text, COMPILED_PATTERNS['ref_header_patterns'])
    if header_idx is not None:
        header_idx2 = find_last_reference_header(text[:header_idx].strip(), COMPILED_PATTERNS['ref_header_patterns'])
        if header_idx2 is not None:
            header_idx3 = find_last_reference_header(text[:header_idx2].strip(), COMPILED_PATTERNS['ref_header_patterns'])
            if header_idx3 is not None:
                return text[:header_idx3].strip(), text[header_idx3:].strip()
            return text[:header_idx2].strip(), text[header_idx2:].strip()
        return text[:header_idx].strip(), text[header_idx:].strip()
    ref_start_line = find_reference_start(text)
    if ref_start_line is not None:
        lines = text.splitlines()
        body = '\n'.join(lines[:ref_start_line])
        refs = '\n'.join(lines[ref_start_line:])
        return body.strip(), refs.strip()
    return text.strip(), ''

def get_splits(df: pl.DataFrame) -> pl.DataFrame:
    bodies, refs = [], []
    for raw_text in df['text']:
        main, ref = split_text_and_references(raw_text)
        bodies.append(main)
        refs.append(ref)
    return df.with_columns(pl.Series('body', bodies), pl.Series('ref', refs))

def tidy_extraction(df) -> pl.DataFrame:
    bad_ids = [f'{DOI_LINK}{e}' for e in ['10.5061/dryad', '10.5281/zenodo', '10.6073/pasta']]

    doi_df = (
        df.with_columns(pl.col('body').str.extract_all(r'10\s*\.\s*\d{4,9}\s*/\s*\S+').alias('match'))
          .explode('match')
          .drop_nulls('match')
          .with_columns(
              pl.col('match').str.replace_all(r'\s+', '')
                             .str.replace(r'[^A-Za-z0-9]+$', '')
                             .str.to_lowercase()
                             .alias('dataset_id')
          )
          .group_by('article_id', 'dataset_id')
          .agg('match')
          .with_columns((DOI_LINK + pl.col('dataset_id')).alias('dataset_id'))
    )

    REGEX_IDS = (
        r"(?i)\b(?:"
        r"CHEMBL\d+|"
        r"E-GEOD-\d+|E-PROT-\d+|EMPIAR-\d+|"
        r"ENSBTAG\d+|ENSOARG\d+|"
        r"EPI_ISL_\d{5,}|EPI\d{6,7}|"
        r"HPA\d+|CP\d{6}|IPR\d{6}|PF\d{5}|KX\d{6}|K0\d{4}|"
        r"PRJNA\d+|PRJEB\d+|PXD\d+|SAMN\d+|"
        r"GSE\d+|GSM\d+|GPL\d+|"
        r"E-MTAB-\d+|E-MEXP-\d+|"
        r"PDB\s?\w{4}|HMDB\d+|"
        r"dryad\.[^\s\"<>]+|pasta\/[^\s\"<>]+|"
        r"(?:SRR|SRX|SRP|ERR|DRR|DRX|DRP|ERP|ERX)\d+"
        r")"
    )


    acc_df = (
        df.with_columns(
            pl.col('text').str.extract_all(REGEX_IDS).alias('match')
        )
        .explode('match')
        .drop_nulls('match')
        .with_columns(
            pl.col('match').str.replace_all(r'\s+', '')
                           .str.replace(r'[^A-Za-z0-9]+$', '')
                           .alias('dataset_id')
        )
        .group_by('article_id', 'dataset_id')
        .agg('match')
        .with_columns(
            pl.when(pl.col('dataset_id').str.starts_with('dryad.'))
              .then(f'{DOI_LINK}10.5061/' + pl.col('dataset_id'))
              .otherwise('dataset_id')
              .alias('dataset_id')
        )
        .with_columns(
            pl.when(pl.col('dataset_id').str.starts_with('pasta/'))
              .then(f'{DOI_LINK}10.6073/' + pl.col('dataset_id'))
              .otherwise('dataset_id')
              .alias('dataset_id')
        )
    )

    df = pl.concat([doi_df, acc_df])

    df = (
        df.unique(['article_id', 'dataset_id'])  # CHANGED
          .filter(~pl.col('article_id').str.replace('_','/').str.contains(pl.col('dataset_id').str.split(DOI_LINK).list.last().str.escape_regex()))
          .filter(~pl.col('dataset_id').str.contains(pl.col('article_id').str.replace('_','/').str.escape_regex()))
          .filter(~pl.col('dataset_id').str.contains('figshare', literal=True))
          .filter(~pl.col('dataset_id').is_in(bad_ids))
          .filter(
              pl.when(is_doi_link('dataset_id') &
                      (pl.col('dataset_id').str.split('/').list.last().str.len_chars() < 5))
               .then(False)
               .otherwise(True)
          )
          .with_columns(pl.col('match').list.unique())
    )
    return df

def get_context_window(text: str, substring: str, window: int = 100) -> str:
    idx = text.find(substring)
    if idx == -1:
        raise ValueError
    start = max(idx - window, 0)
    end = min(idx + len(substring) + window, len(text))
    return text[start:end]

def get_window_df(text_df, ids_df):
    df = ids_df.join(text_df, on='article_id')
    windows = []
    for text, match_ids in df.select('text', 'match').rows():
        windows.append(get_context_window(text, match_ids[0]))
    return df.with_columns(pl.Series('window', windows)).select('article_id', 'dataset_id', 'window')

def main():
    text_df = get_df('/tmp/train_parse')
    df = get_splits(text_df)
    df = get_splits(text_df)
    df.write_parquet("/tmp/train_parse_split.parquet")  # <-- Add this line
    df = tidy_extraction(df)
    df = get_window_df(text_df, df)
    df.write_parquet('/tmp/extracted.parquet')
    df = assume_type(df)
    df.select(['article_id', 'dataset_id', 'type']).with_row_index(name='row_id').write_csv('/kaggle/working/submission.csv')
    if not IS_KAGGLE_SUBMISSION:
        results = evaluate(df)
        for r in results: l.info(r)
        results = evaluate(df, on=['article_id', 'dataset_id', 'type'])
        for r in results: l.info(r)

if __name__=='__main__': main()

Writing /tmp/src/getid.py


In [9]:
%%writefile /tmp/src/addon_fix_broken_dois.py
import re
import polars as pl
from helpers import *

l = get_logger()

BROKEN_DOI_REGEX = re.compile(r'10\s*\.\s*\d{4,9}\s*/\s*[A-Za-z0-9._;()/:\-]+', re.IGNORECASE)

def normalize_doi(s: str) -> str:
    return re.sub(r'[^A-Za-z0-9./-]+', '', s).lower().rstrip('.;:,')

def extract_broken_dois(df: pl.DataFrame) -> pl.DataFrame:
    l.info("Extracting broken DOIs from ref section")
    return (
        df.select(['article_id', 'ref'])
          .with_columns(
              pl.col("ref").map_elements(lambda x: BROKEN_DOI_REGEX.findall(x or ''), return_dtype=pl.List(pl.String)).alias("match")
          )
          .explode("match")
          .drop_nulls("match")
          .with_columns(
              pl.col("match").map_elements(normalize_doi, return_dtype=pl.String).alias("dataset_id")
          )
          .filter(pl.col("dataset_id").str.starts_with("10."))
          .group_by("article_id", "dataset_id")
          .agg(pl.col("match"))
          .with_columns((DOI_LINK + pl.col("dataset_id")).alias("dataset_id"))
    )

def main():
    df = pl.read_parquet('/tmp/train_parse_split.parquet')
    broken_dois = extract_broken_dois(df)
    broken_dois.write_parquet('/tmp/addon_broken_dois.parquet')
    l.info(f"Extracted {broken_dois.shape[0]} broken DOIs")

if __name__ == "__main__":
    main()

Writing /tmp/src/addon_fix_broken_dois.py


In [10]:
%%writefile /tmp/src/addon_reference_priority.py
import polars as pl
from helpers import *

l = get_logger()

def extract_reference_dois(df: pl.DataFrame) -> pl.DataFrame:
    l.info("Tagging DOIs from reference section only")
    return (
        df.select(['article_id', 'ref'])
          .with_columns(
              pl.col("ref")
              .str.extract_all(r'10\.\d{4,9}/\S+')
              .alias("match")
          )
          .explode("match")
          .drop_nulls("match")
          .with_columns(
              pl.col("match")
              .str.replace_all(r'[\s]+', '')
              .str.replace(r'[^A-Za-z0-9./-]+$', '')
              .str.to_lowercase()
              .alias("dataset_id")
          )
          .filter(pl.col("dataset_id").str.starts_with("10."))
          .group_by("article_id", "dataset_id")
          .agg(pl.col("match"))
          .with_columns((DOI_LINK + pl.col("dataset_id")).alias("dataset_id"))
    )

def main():
    df = pl.read_parquet('/tmp/train_parse_split.parquet')
    ref_dois = extract_reference_dois(df)
    ref_dois.write_parquet('/tmp/addon_ref_priority.parquet')
    l.info(f"Extracted {ref_dois.shape[0]} reference-priority DOIs")

if __name__ == "__main__":
    main()

Writing /tmp/src/addon_reference_priority.py


In [11]:
%%writefile /tmp/src/merge_augmented_ids.py
import polars as pl

def drop_if_exists(df: pl.DataFrame, col: str) -> pl.DataFrame:
    return df.drop(col) if col in df.columns else df

def align_columns(df: pl.DataFrame, target_columns: list[str]) -> pl.DataFrame:
    for col in target_columns:
        if col not in df.columns:
            df = df.with_columns(pl.lit(None).alias(col))
    return df.select(target_columns)

# Standard column order
target_columns = ["article_id", "dataset_id", "window"]

# Load base
a = pl.read_parquet("/tmp/scibert_predictions.parquet")
a = align_columns(a, target_columns)

# Load and align broken DOIs
b = pl.read_parquet("/tmp/addon_broken_dois.parquet")
b = drop_if_exists(b, "match")
b = align_columns(b, target_columns)

# Merge a + b
merged_ab = pl.concat([a, b]).unique(subset=["article_id", "dataset_id"])
merged_ab.write_parquet("/tmp/scibert_predictions.parquet")

# Load and align ref-section DOIs
c = pl.read_parquet("/tmp/addon_ref_priority.parquet")
c = drop_if_exists(c, "match")
c = align_columns(c, target_columns)

# Final merge
final = pl.concat([merged_ab, c]).unique(subset=["article_id", "dataset_id"])
final = final.filter(pl.col("window").is_not_null())
final.write_parquet("/tmp/scibert_predictions.parquet")

Writing /tmp/src/merge_augmented_ids.py


In [12]:
%%writefile /tmp/src/llm_validate.py
import polars as pl
import os

from helpers import *

l = get_logger()

SYS_PROMPT_PRIMARY_SECONDARY = """
You are classifying dataset usage in scientific papers. Given a text snippet containing a dataset identifier, determine:

A) Primary - The dataset was created/generated within this study
B) Secondary - An existing dataset was reused/reanalyzed in this study

Key indicators:

PRIMARY (A):
- "we collected", "we generated", "we sequenced"
- "deposited", "uploaded", "submitted" (by authors)
- "our dataset", "newly generated", "experimental data"
- Accession numbers for NEW submissions

SECONDARY (B):
- "previously published", "publicly available", "obtained from"
- "downloaded from", "retrieved from", "reanalyzed"  
- Author citations (Smith et al.)
- "described by", "reported in"

Examples:
1) "RNA-seq data was deposited in GEO under accession GSE123456." → A
2) "We reanalyzed data from GSE789012 (Smith et al., 2020)." → B
3) "Raw sequencing data is available at Zenodo (DOI: 10.5281/zenodo.123)." → A
4) "Expression data downloaded from ArrayExpress E-MTAB-456." → B

Output exactly one letter: A or B.
"""

SYS_PROMPT_CLASSIFY_ALL = """
You are a highly accurate dataset identifier classifier. Given a snippet of academic text containing a DOI or accession number, choose:

  A) Data — the identifier points directly to research data in a repository  
  B) Literature — the identifier points to a journal article, book chapter, protocol paper, or other non-data resource  

=== Repository Prefixes (Always DATA) ===
DOIs starting with:
  • 10.5061 (Dryad)  
  • 10.5281 (Zenodo)  
  • 10.6084 (Figshare)  
  • 10.24433/ (Mendeley Data)  
  • 10.17632 (Mendeley Data)  

Accession numbers:
  • SRA/E- (e.g. SRP, SRA, ERR, DRR)  
  • PRJNA, PRJEB, PRJDB (NCBI BioProject)  
  • SAMN, SAMEA, SAMD (BioSample)
  • GSE, GDS, GPL, GSM (GEO)
  • PXD (PRIDE Proteomics)  
  • E-MTAB, E-GEOD, E-PROT (ArrayExpress)
  • CHEMBL (ChEMBL)
  • EMPIAR (Electron Microscopy)
  • EPI_ISL, EPI (GISAID)
  • HPA (Human Protein Atlas)
  • CP (Cell Painting)
  • IPR, PF (InterPro/Pfam)
  • KX, K0 (KEGG)

=== Publisher DOIs (Usually LITERATURE) ===
  • 10.1038, 10.1126, 10.1016, 10.1007, 10.1101, 10.1371, 10.1073, etc.

=== Few-Shot Examples ===
1) "Raw images are stored on Figshare (DOI 10.6084/m9.figshare.1234567)." → A  
2) "Sequence reads available under BioProject accession PRJNA765432." → A  
3) "As described in Nature Methods (DOI 10.1038/s41592-020-0793-2)." → B  
4) "See Supplementary Data at Zenodo (10.5281/zenodo.987654)." → A  
5) "Method details published in J. Proteome Res. DOI: 10.1021/acs.jproteome.0c00845." → B  
6) "Data has been uploaded to Dryad (10.5061/dryad.x1y2z3)." → A  
7) "Referenced paper: DOI 10.1101/2020.01.01.123456 (bioRxiv preprint)." → B  
8) "Metabolomics data in MetaboLights MTBLS1234." → A  
9) "RNA-seq data deposited in GEO under accession GSE123456." → A
10) "Protein sequences from CHEMBL4567890 were analyzed." → A
11) "Sample metadata available as SAMN12345678." → A

=== Instructions ===
- Use both the identifier and its surrounding context.  
- If the identifier matches known data repository patterns above, choose A.  
- If it's a publisher DOI (journal article), choose B.  
- Look for context clues: "deposited", "uploaded", "archived", "available at" → likely A
- Look for context clues: "described in", "published in", "see methods" → likely B
- Output exactly one letter: A or B, and nothing else.
""".strip()

def build_df():
    """Load extracted data and separate DOIs from accession numbers, but process both"""
    df = pl.read_parquet('/tmp/extracted.parquet')
    
    # Separate but don't filter - we'll process both types
    accession_df = df.filter(~is_doi_link('dataset_id'))
    doi_df = df.filter(is_doi_link('dataset_id'))
    
    # Still write accession subset for backwards compatibility if needed
    accession_df.select('article_id', 'dataset_id').write_csv('/tmp/accid_sub.csv')
    
    # Return the full dataframe for LLM processing
    return df

def build_prompt(tokenizer, df):
    """Build prompts for both DOIs and accession numbers"""
    prompts = []
    for dataset_id, text in df.select('dataset_id', 'window').rows():
        # Enhanced context for the prompt
        enhanced_text = f"Text snippet: {text}\n\nIdentifier to classify: {dataset_id}"
        messages = [
            {'role': 'system', 'content': SYS_PROMPT_CLASSIFY_ALL}, 
            {'role': 'user', 'content': enhanced_text}
        ]
        prompts.append(tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False, enable_thinking=False))
    return df.with_columns(pl.Series('prompt', prompts))

if __name__=='__main__':
    os.environ["VLLM_USE_V1"] = "0"
    import vllm
    from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor
    
    model_path = "/kaggle/input/qwen-3/transformers/32b-awq/1"
    llm = vllm.LLM(model_path, quantization='awq', tensor_parallel_size=2, gpu_memory_utilization=0.9, trust_remote_code=True, dtype="half", enforce_eager=True, max_model_len=2048, disable_log_stats=True, disable_custom_all_reduce=True, enable_prefix_caching=True, task='generate')
    tokenizer = llm.get_tokenizer()
    
    # Process ALL identifiers (both DOIs and accession numbers)
    df = build_df()
    df = build_prompt(tokenizer, df)
    prompts = df['prompt'].to_list()
    
    mclp = MultipleChoiceLogitsProcessor(tokenizer, choices=["A", "B"])
    outputs = llm.generate(prompts, vllm.SamplingParams(seed=777, temperature=0.1, skip_special_tokens=True, max_tokens=1, logits_processors=[mclp], logprobs=len(mclp.choices)), use_tqdm=True)
    
    logprobs = [{lp.decoded_token: lp.logprob for lp in list(lps)} for lps in [output.outputs[0].logprobs[0].values() for output in outputs]]
    pred_letters = [max(lpdict, key=lpdict.get) for lpdict in logprobs]  # "A" or "B"
    type_llm = [True if p == "A" else False for p in pred_letters]
    
    # Add all columns to your dataframe
    df = df.with_columns([
        pl.Series('type_llm', type_llm),
        pl.Series('logprob_A', [d.get("A", float('-inf')) for d in logprobs]),
        pl.Series('logprob_B', [d.get("B", float('-inf')) for d in logprobs]),
        pl.Series('pred_letter', pred_letters)
    ])
    
    # Save full result for hybrid work (now includes both DOIs and accessions)
    df.select(['article_id', 'dataset_id', 'type_llm', 'logprob_A', 'logprob_B']).write_csv('/kaggle/working/submission_llm_logprobs.csv')

    # Filter for data predictions (A)
    choices = [max(d, key=d.get) for d in logprobs]
    types = {'A': True, 'B': False}
    choices = [types[c] for c in choices]
    df = df.with_columns(pl.Series('type', choices))
    
    # Get all data predictions (both DOIs and accessions)
    data_predictions = df.filter(pl.col('type'))
    
    # Apply type classification and write final submission
    df_final = assume_type(data_predictions)
    df_final.select(['article_id', 'dataset_id', 'type']).with_row_index(name='row_id').write_csv('/kaggle/working/submission.csv')
    
    # Evaluation (if not submission)
    if not IS_KAGGLE_SUBMISSION:
        results = evaluate(df_final)
        for r in results: l.info(r) 
        results = evaluate(df_final, on=['article_id', 'dataset_id', 'type'])
        for r in results: l.info(r)

Writing /tmp/src/llm_validate.py


In [13]:
%%writefile /tmp/src/llm_validate-bak.py
import polars as pl
import os

from helpers import *

l = get_logger()

SYS_PROMPT_CLASSIFY_DOI = """
You are a highly accurate DOI/type classifier. Given a snippet of academic text containing a DOI or accession, choose:

  A) Data — the identifier points directly to research data in a repository  
  B) Literature — the identifier points to a journal article, book chapter, protocol paper, or other non-data resource  

=== Repository Prefixes ===
Treat as DATA if the DOI starts with any of:
  • 10.5061 (Dryad)  
  • 10.5281 (Zenodo)  
  • 10.6084 (Figshare)  
  • 10.24433/ (Mendeley Data)  
  • 10.17632 (Mendeley Data)  
  • SRA/E- (e.g. SRP, SRA)  
  • PRJNA, PRJEB, PRJDB (NCBI BioProject)  
  • PRIDE:PXD (Proteomics)  
  • EMBL:E-MTAB, E- (ArrayExpress)  

Everything else is LITERATURE unless you see explicit data-repository context (e.g. “deposited in Dryad under DOI…”).

=== Few-Shot Examples ===
1) “Raw images are stored on Figshare (DOI 10.6084/m9.figshare.1234567).” → A  
2) “Sequence reads available under BioProject accession PRJNA765432.” → A  
3) “As described in Nature Methods (DOI 10.1038/s41592-020-0793-2).” → B  
4) “See Supplementary Data at Zenodo (10.5281/zenodo.987654).” → A  
5) “Method details published in J. Proteome Res. DOI: 10.1021/acs.jproteome.0c00845.” → B  
6) “Data has been uploaded to Dryad (10.5061/dryad.x1y2z3).” → A  
7) “Referenced paper: DOI 10.1101/2020.01.01.123456 (bioRxiv preprint).” → B  
8) “Metabolomics data in MetaboLights MTBLS1234.” → A  

=== Instructions ===
- Use only the identifier itself and its context.  
- If the DOI prefix is in the list above, always choose A.  
- If it belongs to a known publisher prefix (e.g. 10.1007, 10.1038, 10.1126, 10.1016…), choose B.  
- Otherwise, rely on context words (“deposited”, “uploaded”, “archived”) to decide.  
- Output exactly one letter: A or B, and nothing else.
""".strip()

def build_df():
    df = pl.read_parquet('/tmp/extracted.parquet')
    df.filter(~is_doi_link('dataset_id')).select('article_id', 'dataset_id').write_csv('/tmp/accid_sub.csv')
    return df.filter(is_doi_link('dataset_id'))

def build_prompt(tokenizer, df):
    prompts = []
    for doi, text in df.select('dataset_id', 'window').rows():
        messages = [{'role':'system','content': SYS_PROMPT_CLASSIFY_DOI}, {'role':'user', 'content': text}]
        prompts.append(tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False, enable_thinking=False))
    return df.with_columns(pl.Series('prompt', prompts))

if __name__=='__main__':
    os.environ["VLLM_USE_V1"] = "0"
    import vllm
    from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor
    model_path = "/kaggle/input/qwen-3/transformers/32b-awq/1"
    llm = vllm.LLM(model_path, quantization='awq', tensor_parallel_size=2, gpu_memory_utilization=0.9, trust_remote_code=True, dtype="half", enforce_eager=True, max_model_len=2048, disable_log_stats=True, disable_custom_all_reduce=True, enable_prefix_caching=True, task='generate')
    tokenizer = llm.get_tokenizer()
    df = build_df()
    df = build_prompt(tokenizer, df)
    prompts = df['prompt'].to_list()
    mclp = MultipleChoiceLogitsProcessor(tokenizer, choices=["A", "B"])
    outputs = llm.generate(prompts, vllm.SamplingParams(seed=777, temperature=0.1, skip_special_tokens=True, max_tokens=1, logits_processors=[mclp], logprobs=len(mclp.choices)), use_tqdm=True)
    logprobs = [{lp.decoded_token: lp.logprob for lp in list(lps)} for lps in [output.outputs[0].logprobs[0].values() for output in outputs]]
    pred_letters = [max(lpdict, key=lpdict.get) for lpdict in logprobs]  # "A" or "B"
    type_llm = [True if p == "A" else False for p in pred_letters]
    
    # Add all columns to your dataframe
    df = df.with_columns([
        pl.Series('type_llm', type_llm),
        pl.Series('logprob_A', [d.get("A", float('-inf')) for d in logprobs]),
        pl.Series('logprob_B', [d.get("B", float('-inf')) for d in logprobs]),
        pl.Series('pred_letter', pred_letters)
    ])
    
    # Save full result for hybrid work
    df.select(['article_id', 'dataset_id', 'type_llm', 'logprob_A', 'logprob_B']).write_csv('/kaggle/working/submission_llm_logprobs.csv')

    choices = [max(d, key=d.get) for d in logprobs]
    types = {'A': True, 'B': False}
    choices = [types[c] for c in choices]
    df = df.with_columns(pl.Series('type', choices))
    df.filter(pl.col('type')).select('article_id', 'dataset_id').write_csv('/tmp/doi_sub.csv')
    df = pl.concat([pl.read_csv('/tmp/doi_sub.csv'), pl.read_csv('/tmp/accid_sub.csv')])
    df = assume_type(df)
    df.select(['article_id', 'dataset_id', 'type']).with_row_index(name='row_id').write_csv('/kaggle/working/submission.csv')
    if not IS_KAGGLE_SUBMISSION:
        results = evaluate(df)
        for r in results: l.info(r) 
        results = evaluate(df, on=['article_id', 'dataset_id', 'type'])
        for r in results: l.info(r)

Writing /tmp/src/llm_validate-bak.py


In [14]:
%%writefile /tmp/src/post_filter.py
import polars as pl
from helpers import *

"""
Fourth essence: Post-filter to cut FP DOIs that look like literature.
- Read /kaggle/working/submission.csv (output of llm_validate.py)
- Join with /tmp/extracted.parquet to get context window
- Drop DOI rows that (1) start with typical publisher prefixes AND (2) have no data-ish words nearby
- Keep accessions untouched
"""

l = get_logger()

PAPER_PREFIXES = [
    "10.1007", "10.1002", "10.1016", "10.1021", "10.1038", "10.1056",
    "10.1073", "10.1080", "10.1093", "10.1101", "10.1186", "10.1371",
    "10.1111", "10.5194", "10.3390", "10.1126"
]

CONTEXT_RE = r"(?i)\b(data(?:set)?|repository|archive|deposited|available|supplementary|raw(?:\s+data)?|uploaded|hosted|stored|accession)\b"

def is_paper_prefix(col: str = "dataset_id") -> pl.Expr:
    expr = pl.lit(False)
    for p in PAPER_PREFIXES:
        expr = expr | pl.col(col).str.starts_with(f"{DOI_LINK}{p}")
    return expr

def main():
    sub = pl.read_csv("/kaggle/working/submission.csv")

    # Normalize columns: drop row_id if present so concat widths match
    if "row_id" in sub.columns:
        sub = sub.drop("row_id")

    # Context windows
    win = pl.read_parquet("/tmp/extracted.parquet").select("article_id", "dataset_id", "window")

    # DOI & ACC split
    doi_rows = sub.filter(is_doi_link("dataset_id")).join(win, on=["article_id", "dataset_id"], how="left")
    acc_rows = sub.filter(~is_doi_link("dataset_id"))

    keep_mask = (
        (~is_paper_prefix("dataset_id"))  # not a known paper prefix
        | doi_rows["window"].fill_null("").str.contains(CONTEXT_RE)
    )

    kept_doi = doi_rows.filter(keep_mask).select("article_id", "dataset_id", "type")
    final = pl.concat([kept_doi, acc_rows.select("article_id", "dataset_id", "type")])

    # Re-eval & save
    if not IS_KAGGLE_SUBMISSION:
        for r in evaluate(final): l.info(r)
        for r in evaluate(final, on=["article_id", "dataset_id", "type"]): l.info(r)

    final.with_row_index("row_id").write_csv("/kaggle/working/submission.csv")

if __name__ == "__main__":
    main()

Writing /tmp/src/post_filter.py


In [15]:
%%writefile /tmp/src/post_filter_scibert.py
import polars as pl
from helpers import *

"""
Fourth essence: Post-filter to cut FP DOIs that look like literature.
- Read /kaggle/working/submission.csv (output of llm_validate.py)
- Join with /tmp/extracted.parquet to get context window
- Drop DOI rows that (1) start with typical publisher prefixes AND (2) have no data-ish words nearby
- Keep accessions untouched
"""

l = get_logger()

PAPER_PREFIXES = [
    "10.1007", "10.1002", "10.1016", "10.1021", "10.1038", "10.1056",
    "10.1073", "10.1080", "10.1093", "10.1101", "10.1186", "10.1371",
    "10.1111", "10.5194", "10.3390", "10.1126"
]

CONTEXT_RE = r"(?i)\b(data(?:set)?|repository|archive|deposited|available|supplementary|raw(?:\s+data)?|uploaded|hosted|stored|accession)\b"

def is_paper_prefix(col: str = "dataset_id") -> pl.Expr:
    expr = pl.lit(False)
    for p in PAPER_PREFIXES:
        expr = expr | pl.col(col).str.starts_with(f"{DOI_LINK}{p}")
    return expr

def main():
    sub = pl.read_csv("/kaggle/working/submission.csv")

    # Normalize columns: drop row_id if present so concat widths match
    if "row_id" in sub.columns:
        sub = sub.drop("row_id")

    # Context windows
    win = pl.read_parquet("/tmp/scibert_predictions.parquet").select("article_id", "dataset_id", "window",'r_type')

    # DOI & ACC split
    doi_rows = sub.filter(is_doi_link("dataset_id")).join(win, on=["article_id", "dataset_id"], how="left")
    acc_rows = sub.filter(~is_doi_link("dataset_id"))

    keep_mask = (
        (~is_paper_prefix("dataset_id"))  # not a known paper prefix
        | doi_rows["window"].fill_null("").str.contains(CONTEXT_RE)
    )

    kept_doi = doi_rows.filter(keep_mask).select("article_id", "dataset_id", "r_type", 'type')
    acc_rows = acc_rows.join(win, on=["article_id", "dataset_id"], how="left")
    final = pl.concat([kept_doi, acc_rows.select("article_id", "dataset_id", "r_type", 'type')])
    
    # Replace 'type' with 'r_type' for submission
    final = final.drop("type").rename({"r_type": "type"})
    
    # Re-eval & save
    if not IS_KAGGLE_SUBMISSION:
        for r in evaluate(final): l.info(r)
        for r in evaluate(final, on=["article_id", "dataset_id", "type"]): l.info(r)
    
    final.with_row_index("row_id").write_csv("/kaggle/working/submission_scibert.csv")


if __name__ == "__main__":
    main()

Writing /tmp/src/post_filter_scibert.py


In [16]:
%cd /tmp
! LOG_LEVEL=INFO python src/parse.py /tmp/train_parse
! python src/check_parse.py

/tmp


In [17]:
! python src/getid.py

Traceback (most recent call last):
  File "/tmp/src/getid.py", line 196, in <module>
    if __name__=='__main__': main()
                             ^^^^^^
  File "/tmp/src/getid.py", line 188, in main
    df = assume_type(df)
         ^^^^^^^^^^^^^^^
  File "/tmp/src/helpers.py", line 99, in assume_type
    types = [classify_type(w, d) for w, d in df.select(['window', 'dataset_id']).rows()]
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/src/helpers.py", line 99, in <listcomp>
    types = [classify_type(w, d) for w, d in df.select(['window', 'dataset_id']).rows()]
             ^^^^^^^^^^^^^^^^^^^
  File "/tmp/src/helpers.py", line 75, in classify_type
    if pd.isna(window):
       ^^
NameError: name 'pd' is not defined. Did you mean: 'pl'?


In [18]:
%%writefile /tmp/src/SciBert_train.py
import pandas as pd
import torch
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    TrainerCallback,
    EarlyStoppingCallback
)
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ["WANDB_DISABLED"] = "true"
# Load data from parquet and CSV
extracted_df = pd.read_parquet("/tmp/extracted.parquet")
labels_df = pd.read_csv("/kaggle/input/make-data-count-finding-data-references/train_labels.csv")

# Normalize keys for join
labels_df["article_id"] = labels_df["article_id"].str.replace("/", "_")
labels_df["dataset_id"] = labels_df["dataset_id"].str.lower()

extracted_df["dataset_id"] = extracted_df["dataset_id"].str.lower()

# Merge and drop rows without labels
df = extracted_df.merge(labels_df, on=["article_id", "dataset_id"], how="inner")
df = df.dropna(subset=["window", "type"])

# Label encoding
type_to_label = {"Primary": 0, "Secondary": 1} #, "Missing": 2}
df["label"] = df["type"].map(type_to_label)

# Train/val split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df["window"].tolist(),
    df["label"].tolist(),
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)

# Tokenization
model_name = "allenai/scibert_scivocab_uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

# Dataset wrapper
class DataRelationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = DataRelationDataset(train_encodings, train_labels)
val_dataset = DataRelationDataset(val_encodings, val_labels)

# Model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, 
                                                           hidden_dropout_prob=0.3,      # Default is usually 0.1
                                                           attention_probs_dropout_prob=0.3,  # Default is usually 0.1
)

# Metrics
def compute_metrics(p):    
    preds = p.predictions.argmax(-1)
    f1_micro = f1_score(p.label_ids, preds, average='micro')
    f1_macro = f1_score(p.label_ids, preds, average='macro')
    report = classification_report(p.label_ids, preds, output_dict=True, zero_division=0)

    def safe_f1(class_label):
        return report[str(class_label)]["f1-score"] if str(class_label) in report else 0.0

    return {
        "f1_micro": f1_micro,
        "f1_macro": f1_macro,
        "f1_primary": safe_f1(0),
        "f1_secondary": safe_f1(1),
        "f1_missing": safe_f1(2)
    }

# Training setup
training_args = TrainingArguments(
    output_dir="/kaggle/working/results",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=101,
    learning_rate=.5e-5,
    weight_decay=0.15, # Increase from 0.01 to 0.1 or even 0.2
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1_macro",
    greater_is_better=True,
    save_total_limit = 4,
    fp16=True
)

class PrintDevicesCallback(TrainerCallback):
    def on_train_begin(self, args, state, control, **kwargs):
        print(f"Training on {torch.cuda.device_count()} GPUs:", [torch.cuda.get_device_name(i) for i in range(torch.cuda.device_count())])

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train
trainer.train()
model.save_pretrained("/kaggle/working/best_model")
tokenizer.save_pretrained("/kaggle/working/best_model")
! zip -r /kaggle/working/output-tbt-2L.zip /kaggle/working/best_model
! cd /kaggle/working
from IPython.display import FileLink
FileLink(r'output-tbt-2L.zip')

Writing /tmp/src/SciBert_train.py


In [19]:
# %%writefile /tmp/src/SciBert_infer.py
from tqdm import tqdm
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
# 🧠 Load model and tokenizer
model_path = "/kaggle/input/btb-2l-scibert/kaggle/working/best_model"
token_path = model_path

tokenizer = AutoTokenizer.from_pretrained(token_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 🔮 Predict type
label_map = {0: "Primary", 1: "Secondary",} # 2: 'Missing'}
batch_size = 8
preds = []
rows = pd.read_parquet("/tmp/extracted.parquet")

# Drop NA windows
rows = rows.dropna(subset=["window"]).reset_index(drop=True)

logits_list = []

for i in tqdm(range(0, len(rows), batch_size)):
    batch_texts = rows["window"].iloc[i:i+batch_size].tolist()
    enc = tokenizer(batch_texts, truncation=True, padding=True, max_length=512, return_tensors="pt").to(device)
    with torch.no_grad():
        logits = model(**enc).logits
        p = torch.argmax(logits, dim=1).cpu().tolist()
        preds.extend(p)
        logits_list.extend(logits.cpu().tolist())


logits_df = pd.DataFrame(logits_list, columns=["logit_primary", "logit_secondary"])
rows = pd.concat([rows.reset_index(drop=True), logits_df], axis=1)
label_map = {0: "Primary", 1: "Secondary"}

# preds is a list of ints from argmax
rows["pred_label"] = [label_map[p] for p in preds]

# Then export
rows[["article_id", "dataset_id", "pred_label", "logit_primary", "logit_secondary"]].to_csv(
    "/kaggle/working/submission_scibert_logits.csv", index=False
)
rows[["article_id", "dataset_id", "pred_label", "logit_primary", "logit_secondary"]].to_csv("/kaggle/working/submission_scibert_logits.csv", index=False)
rows["r_type"] = [label_map.get(p, "Missing") for p in preds]
rows[["article_id", "dataset_id", 'window',"r_type"]].to_parquet("/tmp/scibert_predictions.parquet")

2025-08-03 13:09:27.434898: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754226567.772960      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754226567.869828      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
100%|██████████| 3/3 [00:00<00:00,  7.62it/s]


In [20]:
! python src/post_filter_scibert.py
#! rm /kaggle/working/submission.csv
#! mv /kaggle/working/submission_scibert.csv /kaggle/working/submission.csv

Traceback (most recent call last):
  File "/tmp/src/post_filter_scibert.py", line 63, in <module>
    main()
  File "/tmp/src/post_filter_scibert.py", line 29, in main
    sub = pl.read_csv("/kaggle/working/submission.csv")
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/polars/_utils/deprecation.py", line 92, in wrapper
    return function(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/polars/_utils/deprecation.py", line 92, in wrapper
    return function(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/polars/_utils/deprecation.py", line 92, in wrapper
    return function(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/polars/io/csv/functions.py", line 534, in read_csv
    df = _read_csv_impl(
         ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pola

In [21]:
#! python src/addon_fix_broken_dois.py
#! python src/addon_reference_priority.py
#! python src/merge_augmented_ids.py  # or insert the merging logic inline

In [22]:
! python src/llm_validate.py
! python src/post_filter.py

2025-08-03 13:09:54.398109: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754226594.420327     181 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754226594.427004     181 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
INFO 08-03 13:09:58 [__init__.py:244] Automatically detected platform cuda.
INFO 08-03 13:10:15 [config.py:1472] Using max model len 2048
WARNING 08-03 13:10:15 [config.py:960] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
WARNING 08-03 13:10:16 [cuda.py:102] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot b

In [23]:
# %%writefile /tmp/src/HybridProbs.py
import pandas as pd

# Load everything
truth = pd.read_csv("/kaggle/input/make-data-count-finding-data-references/train_labels.csv")
truth = truth[truth["type"] != "Missing"].copy()
llm = pd.read_csv("/kaggle/working/submission_llm_logprobs.csv").rename(columns={"type": "type_llm"})
scibert = pd.read_csv("/kaggle/working/submission_scibert_logits.csv").rename(columns={"pred_label": "type_scibert"})
# Ensure no trailing slashes, lowercase, and consistent formats
truth["article_id"] = truth["article_id"].astype(str).str.strip()
truth["dataset_id"] = truth["dataset_id"].astype(str).str.strip().str.lower()

llm["article_id"] = llm["article_id"].astype(str).str.strip()
llm["dataset_id"] = llm["dataset_id"].astype(str).str.strip().str.lower()

scibert["article_id"] = scibert["article_id"].astype(str).str.strip()
scibert["dataset_id"] = scibert["dataset_id"].astype(str).str.strip().str.lower()

# Merge
df = (
    truth.rename(columns={"type": "type_truth"})
    .merge(llm, on=["article_id", "dataset_id"], how="left")
    .merge(scibert, on=["article_id", "dataset_id"], how="left")
)

# Save
df.to_csv("/kaggle/working/merged_predictions_with_confidence.csv", index=False)

# Preview
df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,article_id,dataset_id,type_truth,type_llm,logprob_A,logprob_B,type_scibert,logit_primary,logit_secondary
0,10.1002_2017jc013030,https://doi.org/10.17882/49388,Primary,NaN,NaN,NaN,NaN,NaN,NaN
1,10.1002_ece3.4466,https://doi.org/10.5061/dryad.r6nq870,Primary,True,0.0,-204.424896,Primary,2.372571,-1.902298
2,10.1002_ece3.5260,https://doi.org/10.5061/dryad.2f62927,Primary,True,0.0,-199.736267,Primary,2.089813,-1.519837
3,10.1002_ece3.6144,https://doi.org/10.5061/dryad.zw3r22854,Primary,True,0.0,-193.484772,Primary,2.192287,-1.654931
4,10.1002_ece3.6303,https://doi.org/10.5061/dryad.37pvmcvgb,Primary,True,0.0,-190.984161,Primary,2.379811,-1.916410


In [24]:
#! python src/HybridProbs.py

In [25]:
#%%writefile /tmp/src/HybridProbsMerge.py

truth_keys = set(zip(truth["article_id"], truth["dataset_id"]))
llm_keys = set(zip(llm["article_id"], llm["dataset_id"]))
scibert_keys = set(zip(scibert["article_id"], scibert["dataset_id"]))

print("truth total:", len(truth_keys))
print("truth ∩ llm:", len(truth_keys & llm_keys))
print("truth ∩ scibert:", len(truth_keys & scibert_keys))
missing_in_llm = truth_keys - llm_keys
missing_in_scibert = truth_keys - scibert_keys

print("Missing in LLM:", list(missing_in_llm)[:5])
print("Missing in SciBERT:", list(missing_in_scibert)[:5])
import numpy as np

def softmax_probs_llm(logA, logB):
    logits = np.array([logA, logB])
    probs = np.exp(logits - np.max(logits))  # logit-stabilized
    probs /= probs.sum()
    return probs[0], probs[1]  # P(A), P(B)
llm_df = llm
llm_df["llm_prob_A"], llm_df["llm_prob_B"] = zip(*llm_df.apply(
    lambda row: softmax_probs_llm(row["logprob_A"], row["logprob_B"]),
    axis=1
))
llm_df["llm_margin"] = abs(llm_df["llm_prob_A"] - llm_df["llm_prob_B"])
scibert_df = scibert.copy()
import numpy as np

def softmax_probs_scibert_2class(logit_primary, logit_secondary):
    logits = np.array([logit_primary, logit_secondary])
    exps = np.exp(logits - np.max(logits))  # for numerical stability
    return exps / exps.sum()

# Apply row-wise
scibert_df[["scibert_Primary", "scibert_Secondary"]] = scibert_df.apply(
    lambda row: pd.Series(softmax_probs_scibert_2class(row["logit_primary"], row["logit_secondary"])),
    axis=1
)

# Compute margin
scibert_df["scibert_margin"] = abs(scibert_df["scibert_Primary"] - scibert_df["scibert_Secondary"])
merged_df = df.copy()
merged_df = merged_df.merge(llm_df[["article_id", "dataset_id", "llm_prob_A", "llm_prob_B", "llm_margin"]], on=["article_id", "dataset_id"], how="left")
merged_df = merged_df.merge(scibert_df[["article_id", "dataset_id", "scibert_Primary", "scibert_Secondary", "scibert_margin"]], on=["article_id", "dataset_id"], how="left")
def choose_type(row):
    if row["llm_margin"] >= 0.6:
        return row["type_llm"]
    elif row["scibert_margin"] >= 0.3:
        return row["type_scibert"]
    else:
        return row["type_llm"]  # fallback if both are unsure
merged_df["type_hybrid"] = merged_df.apply(choose_type, axis=1)

##### OR 
def combine_probs(row, alpha=0.6):
    # LLM is binary: A=Primary, B=Secondary
    p_primary = alpha * row["llm_prob_A"] + (1 - alpha) * row["scibert_Primary"]
    p_secondary = alpha * row["llm_prob_B"] + (1 - alpha) * row["scibert_Secondary"]
    return "Primary" if p_primary > p_secondary else "Secondary"

# merged_df["type_hybrid"] = merged_df.apply(combine_probs, axis=1)
merged_df.to_csv("/kaggle/working/merged_predictions_with_confidence.csv", index=False)

truth total: 719
truth ∩ llm: 8
truth ∩ scibert: 8
Missing in LLM: [('10.1136_jitc-2021-003114', 'cvcl_1919'), ('10.5194_essd-2023-198', 'https://doi.org/10.17882/94032'), ('10.1016_j.jlp.2022.104761', 'https://doi.org/10.23642/usn.15134442.v1'), ('10.1371_journal.pone.0212669', 'e-geod-48780'), ('10.1038_s41396-020-00885-8', 'ipr004963')]
Missing in SciBERT: [('10.1136_jitc-2021-003114', 'cvcl_1919'), ('10.5194_essd-2023-198', 'https://doi.org/10.17882/94032'), ('10.1016_j.jlp.2022.104761', 'https://doi.org/10.23642/usn.15134442.v1'), ('10.1371_journal.pone.0212669', 'e-geod-48780'), ('10.1038_s41396-020-00885-8', 'ipr004963')]


In [26]:
%%writefile /tmp/src/nn_train.py

import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm

# Load inputs
llm_df = pd.read_csv("/kaggle/working/submission_llm_logprobs.csv")
scibert_df = pd.read_csv("/kaggle/working/submission_scibert_logits.csv")
extracted_df = pd.read_parquet("/tmp/extracted.parquet")

# Merge and keep only non-Missing
df = (
    extracted_df.merge(llm_df, on=["article_id", "dataset_id"], how="left")
               .merge(scibert_df, on=["article_id", "dataset_id"], how="left")
)
df = df[df["pred_label"] != "Missing"].reset_index(drop=True)

# 🔠 Sentence embeddings
embedder = SentenceTransformer("/kaggle/input/all-minilm-l6-v2-and-deps/all-MiniLM-L6-v2/")
embed_window = embedder.encode(df["window"].tolist(), batch_size=64, show_progress_bar=True)
embed_dsid = embedder.encode(df["dataset_id"].tolist(), batch_size=64, show_progress_bar=True)

# 🔢 One-hot encoding of types
llm_onehot = pd.get_dummies(df["type_llm"], prefix="llm")
scibert_onehot = pd.get_dummies(df["pred_label"], prefix="scibert")
llm_onehot = llm_onehot.reindex(columns=["llm_Primary", "llm_Secondary"], fill_value=0)
scibert_onehot = scibert_onehot.reindex(columns=["scibert_Primary", "scibert_Secondary"], fill_value=0)

# 🧠 Construct training matrix
X = np.hstack([
    embed_window,
    embed_dsid,
    df[["logit_primary", "logit_secondary"]].values,
    df[["logprob_A", "logprob_B"]].values,
    llm_onehot.values,
    scibert_onehot.values
])

# 🎯 Labels (0=Primary, 1=Secondary)
label_map = {"Primary": 0, "Secondary": 1}
y = df["pred_label"].map(label_map).values

# 🧽 Normalize
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 🔀 Train/val split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# 🧱 Dataset and dataloader
train_ds = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train))
val_ds = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val))
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=64)

# 🧠 Define model
class Classifier(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 2)
        )
    def forward(self, x):
        return self.net(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Classifier(input_dim=X.shape[1]).to(device)
opt = torch.optim.Adam(model.parameters(), lr=.5e-5)
loss_fn = nn.CrossEntropyLoss()

# 🔁 Training loop
best_f1 = 0.0
for epoch in range(1, 5000):
    model.train()
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        opt.zero_grad()
        loss = loss_fn(model(xb), yb)
        loss.backward()
        opt.step()

    model.eval()
    preds, true = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(device)
            logits = model(xb)
            p = torch.argmax(logits, dim=1).cpu().numpy()
            preds.extend(p)
            true.extend(yb.numpy())

    f1 = f1_score(true, preds, average="macro")
    print(f"Epoch {epoch+1}: Val F1_macro={f1:.4f}, Loss={loss.item():.3f}")
    if f1 > best_f1:
        print("✅ New best model saved!")
        torch.save(model, "/kaggle/working/best_embed_mlp.pt")
        best_f1 = f1

Writing /tmp/src/nn_train.py


In [27]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report
from tqdm import tqdm
import logging
import os
from pathlib import Path

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def load_and_merge_data():
    """Load and merge all data sources"""
    try:
        llm_df = pd.read_csv("/kaggle/working/submission_llm_logprobs.csv")
        scibert_df = pd.read_csv("/kaggle/working/submission_scibert_logits.csv")
        extracted_df = pd.read_parquet("/tmp/extracted.parquet")
        
        # Merge datasets
        df = (
            extracted_df.merge(llm_df, on=["article_id", "dataset_id"], how="left")
                       .merge(scibert_df, on=["article_id", "dataset_id"], how="left")
        )
        
        # Filter out Missing labels
        df = df[df["pred_label"] != "Missing"].reset_index(drop=True)
        logger.info(f"Loaded {len(df)} samples after filtering")
        
        return df
    except Exception as e:
        logger.error(f"Error loading data: {e}")
        raise

def create_embeddings(df, embedder_path="/kaggle/input/all-minilm-l6-v2-and-deps/all-MiniLM-L6-v2/"):
    """Generate sentence embeddings for window and dataset_id"""
    try:
        embedder = SentenceTransformer(embedder_path)
        
        logger.info("Generating window embeddings...")
        embed_window = embedder.encode(
            df["window"].tolist(), 
            batch_size=64, 
            show_progress_bar=True,
            convert_to_numpy=True
        )
        
        logger.info("Generating dataset_id embeddings...")
        embed_dsid = embedder.encode(
            df["dataset_id"].tolist(), 
            batch_size=64, 
            show_progress_bar=True,
            convert_to_numpy=True
        )
        
        return embed_window, embed_dsid
    except Exception as e:
        logger.error(f"Error creating embeddings: {e}")
        raise

def prepare_features(df, embed_window, embed_dsid):
    """Prepare feature matrix and labels"""
    # One-hot encodings with consistent columns
    llm_onehot = pd.get_dummies(df["type_llm"], prefix="llm")
    scibert_onehot = pd.get_dummies(df["pred_label"], prefix="scibert")
    
    # Ensure consistent columns
    for col in ["llm_Primary", "llm_Secondary"]:
        if col not in llm_onehot.columns:
            llm_onehot[col] = 0
    
    for col in ["scibert_Primary", "scibert_Secondary"]:
        if col not in scibert_onehot.columns:
            scibert_onehot[col] = 0
    
    llm_onehot = llm_onehot[["llm_Primary", "llm_Secondary"]]
    scibert_onehot = scibert_onehot[["scibert_Primary", "scibert_Secondary"]]
    
    # Construct feature matrix
    X = np.hstack([
        embed_window,
        embed_dsid,
        df[["logit_primary", "logit_secondary"]].values,
        df[["logprob_A", "logprob_B"]].values,
        llm_onehot.values,
        scibert_onehot.values
    ])
    
    # Labels
    label_map = {"Primary": 0, "Secondary": 1}
    y = df["pred_label"].map(label_map).values
    
    logger.info(f"Feature matrix shape: {X.shape}")
    logger.info(f"Label distribution: {np.bincount(y)}")
    
    return X, y

class Classifier(nn.Module):
    """Improved classifier with configurable architecture"""
    def __init__(self, input_dim, hidden_dim=128, dropout=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, 2)
        )
    
    def forward(self, x):
        return self.net(x)

def train_epoch(model, train_loader, optimizer, loss_fn, device):
    """Train for one epoch"""
    model.train()
    total_loss = 0
    num_batches = 0
    
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        
        optimizer.zero_grad()
        logits = model(xb)
        loss = loss_fn(logits, yb)
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        total_loss += loss.item()
        num_batches += 1
    
    return total_loss / num_batches

def evaluate(model, val_loader, device):
    """Evaluate model on validation set"""
    model.eval()
    preds, true = [], []
    
    with torch.no_grad():
        for xb, yb in val_loader:
            xb = xb.to(device)
            logits = model(xb)
            p = torch.argmax(logits, dim=1).cpu().numpy()
            preds.extend(p)
            true.extend(yb.numpy())
    
    f1 = f1_score(true, preds, average="macro")
    return f1, preds, true

def main():
    # Set random seeds for reproducibility
    torch.manual_seed(42)
    np.random.seed(42)
    
    # Load data
    df = load_and_merge_data()
    
    # Create embeddings
    embed_window, embed_dsid = create_embeddings(df)
    
    # Prepare features
    X, y = prepare_features(df, embed_window, embed_dsid)
    
    # Normalize features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    
    # Train/validation split
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )
    
    # Create datasets and dataloaders
    train_ds = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train))
    val_ds = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val))
    
    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=64)
    
    # Model setup
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Using device: {device}")
    
    model = Classifier(input_dim=X.shape[1]).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=10
    )
    loss_fn = nn.CrossEntropyLoss()
    
    # Training loop
    best_f1 = 0.0
    patience_counter = 0
    max_patience = 50
    
    logger.info("Starting training...")
    
    for epoch in range(1000):  # Reduced max epochs
        # Train
        avg_loss = train_epoch(model, train_loader, optimizer, loss_fn, device)
        
        # Evaluate
        val_f1, preds, true = evaluate(model, val_loader, device)
        
        # Learning rate scheduling
        scheduler.step(val_f1)
        
        # Logging
        if epoch % 10 == 0:
            logger.info(f"Epoch {epoch}: Val F1={val_f1:.4f}, Loss={avg_loss:.4f}")
        
        # Save best model
        if val_f1 > best_f1:
            best_f1 = val_f1
            patience_counter = 0
            
            # Save model
            save_path = "/kaggle/working/best_embed_mlp.pt"
            torch.save({
                'model_state_dict': model.state_dict(),
                'scaler': scaler,
                'best_f1': best_f1,
                'epoch': epoch
            }, save_path)
            
            logger.info(f"✅ New best model saved! F1: {best_f1:.4f}")
            
            # Print detailed classification report
            if epoch % 50 == 0:
                print("\nClassification Report:")
                print(classification_report(true, preds, target_names=["Primary", "Secondary"]))
        else:
            patience_counter += 1
        
        # Early stopping
        if patience_counter >= max_patience:
            logger.info(f"Early stopping at epoch {epoch}")
            break
    
    logger.info(f"Training completed. Best F1: {best_f1:.4f}")

if __name__ == "__main__":
    main()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Classification Report:
              precision    recall  f1-score   support

     Primary       1.00      0.25      0.40         4
   Secondary       0.25      1.00      0.40         1

    accuracy                           0.40         5
   macro avg       0.62      0.62      0.40         5
weighted avg       0.85      0.40      0.40         5



In [28]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import StandardScaler
import logging
from pathlib import Path

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Paths
LLM_PATH = "/kaggle/working/submission_llm_logprobs.csv"
SCIBERT_PATH = "/kaggle/working/submission_scibert_logits.csv"
EXTRACTED_PATH = "/tmp/extracted.parquet"
MODEL_PATH = "/kaggle/working/best_embed_mlp.pt"
OUTPUT_PATH = "/kaggle/working/submission_nn.csv"

class Classifier(nn.Module):
    """Improved classifier matching training code"""
    def __init__(self, input_dim, hidden_dim=128, dropout=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim // 2, 2)
        )
    
    def forward(self, x):
        return self.net(x)

def load_and_merge_data():
    """Load and merge all data sources"""
    try:
        llm = pd.read_csv(LLM_PATH)
        scibert = pd.read_csv(SCIBERT_PATH)
        extracted = pd.read_parquet(EXTRACTED_PATH)
        
        # Merge datasets
        df = (
            extracted.merge(llm, on=["article_id", "dataset_id"], how="inner")
                     .merge(scibert, on=["article_id", "dataset_id"], how="inner")
        )
        
        # Filter out 'Missing'
        df = df[df["pred_label"] != "Missing"].reset_index(drop=True)
        logger.info(f"Loaded {len(df)} samples for inference")
        
        return df
    except Exception as e:
        logger.error(f"Error loading data: {e}")
        raise

def create_embeddings(df, embedder_path="/kaggle/input/all-minilm-l6-v2-and-deps/all-MiniLM-L6-v2/"):
    """Generate sentence embeddings for window and dataset_id"""
    try:
        embedder = SentenceTransformer(embedder_path)
        
        logger.info("Generating window embeddings...")
        embed_window = embedder.encode(
            df["window"].tolist(), 
            batch_size=64, 
            show_progress_bar=True,
            convert_to_numpy=True
        )
        
        logger.info("Generating dataset_id embeddings...")
        embed_dsid = embedder.encode(
            df["dataset_id"].tolist(), 
            batch_size=64, 
            show_progress_bar=True,
            convert_to_numpy=True
        )
        
        return embed_window, embed_dsid
    except Exception as e:
        logger.error(f"Error creating embeddings: {e}")
        raise

def prepare_features(df, embed_window, embed_dsid):
    """Prepare feature matrix - must match training exactly"""
    # One-hot encodings with consistent columns
    llm_onehot = pd.get_dummies(df["type_llm"], prefix="llm")
    scibert_onehot = pd.get_dummies(df["pred_label"], prefix="scibert")
    
    # Ensure consistent columns (same as training)
    for col in ["llm_Primary", "llm_Secondary"]:
        if col not in llm_onehot.columns:
            llm_onehot[col] = 0
    
    for col in ["scibert_Primary", "scibert_Secondary"]:
        if col not in scibert_onehot.columns:
            scibert_onehot[col] = 0
    
    llm_onehot = llm_onehot[["llm_Primary", "llm_Secondary"]]
    scibert_onehot = scibert_onehot[["scibert_Primary", "scibert_Secondary"]]
    
    # Construct feature matrix (same order as training)
    X = np.hstack([
        embed_window,
        embed_dsid,
        df[["logit_primary", "logit_secondary"]].values,
        df[["logprob_A", "logprob_B"]].values,
        llm_onehot.values,
        scibert_onehot.values
    ])
    
    logger.info(f"Feature matrix shape: {X.shape}")
    return X

def load_model_and_scaler():
    """Load trained model and scaler"""
    try:
        # Load saved checkpoint
        checkpoint = torch.load(MODEL_PATH, map_location='cpu', weights_only=False)
        
        # Check if it's the new format (with scaler) or old format
        if isinstance(checkpoint, dict) and 'model_state_dict' in checkpoint:
            # New format
            scaler = checkpoint['scaler']
            model_state = checkpoint['model_state_dict']
            logger.info(f"Loaded model with F1: {checkpoint.get('best_f1', 'unknown')}")
        else:
            # Old format - model only
            model_state = checkpoint.state_dict() if hasattr(checkpoint, 'state_dict') else None
            scaler = None
            logger.warning("Old model format detected - scaler not saved with model")
        
        return model_state, scaler
    except Exception as e:
        logger.error(f"Error loading model: {e}")
        raise

def main():
    # Load data
    df = load_and_merge_data()
    
    # Create embeddings
    embed_window, embed_dsid = create_embeddings(df)
    
    # Prepare features
    X = prepare_features(df, embed_window, embed_dsid)
    
    # Load model and scaler
    model_state, saved_scaler = load_model_and_scaler()
    
    # Normalize features
    if saved_scaler is not None:
        # Use saved scaler from training
        logger.info("Using saved scaler from training")
        X = saved_scaler.transform(X)
    else:
        # Fallback: fit new scaler (not ideal for inference)
        logger.warning("No saved scaler found - fitting new scaler (may cause issues)")
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
    
    # Setup device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    logger.info(f"Using device: {device}")
    
    # Load model
    model = Classifier(input_dim=X.shape[1])
    
    if model_state is not None:
        model.load_state_dict(model_state)
    else:
        # Fallback for old format
        model = torch.load(MODEL_PATH, map_location=device, weights_only=False)
    
    model.to(device)
    model.eval()
    
    logger.info("Running inference...")
    
    # Predict in batches to handle large datasets
    batch_size = 1000
    all_preds = []
    
    with torch.no_grad():
        for i in range(0, len(X), batch_size):
            batch_X = X[i:i+batch_size]
            inputs = torch.tensor(batch_X, dtype=torch.float32).to(device)
            logits = model(inputs)
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
    
    # Map predictions
    id2label = {0: "Primary", 1: "Secondary"}
    df["type"] = [id2label[p] for p in all_preds]
    
    # Log prediction distribution
    pred_counts = df["type"].value_counts()
    logger.info(f"Prediction distribution: {pred_counts.to_dict()}")
    
    # Create submission
    submission = df[["article_id", "dataset_id", "type"]].copy()
    submission.insert(0, "row_id", submission.index)
    
    # Save submission
    submission.to_csv(OUTPUT_PATH, index=False)
    logger.info(f"✅ Submission saved to {OUTPUT_PATH}")
    logger.info(f"Submission shape: {submission.shape}")
    
    # Display first few rows
    print("\nFirst 5 rows of submission:")
    print(submission.head())

if __name__ == "__main__":
    main()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


First 5 rows of submission:
   row_id              article_id                              dataset_id  \
0       0       10.1002_ecs2.4619         https://doi.org/10.25349/d9qw5x   
1       1       10.1002_ecs2.4619  PASTA/D835832D7FD00D9E4466E44EEA87FAB3   
2       2       10.1002_ece3.4466   https://doi.org/10.5061/dryad.r6nq870   
3       3  10.1002_anie.201916483  https://doi.org/10.1002/ange.201916483   
4       4       10.1002_ece3.5395  https://doi.org/10.5441/001/1.71r7pp6q   

        type  
0    Primary  
1    Primary  
2    Primary  
3    Primary  
4  Secondary  


In [29]:
%%writefile /tmp/src/nn_infer.py
import pandas as pd
import numpy as np
import torch
from torch import nn
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import StandardScaler

# Paths
LLM_PATH = "/kaggle/working/submission_llm_logprobs.csv"
SCIBERT_PATH = "/kaggle/working/submission_scibert_logits.csv"
EXTRACTED_PATH = "/tmp/extracted.parquet"
MODEL_PATH = "/kaggle/working/best_embed_mlp.pt"
OUTPUT_PATH = "/kaggle/working/submission_nn.csv"

# Load data
llm = pd.read_csv(LLM_PATH)
scibert = pd.read_csv(SCIBERT_PATH)
extracted = pd.read_parquet(EXTRACTED_PATH)

# Merge
df = (
    extracted.merge(llm, on=["article_id", "dataset_id"], how="inner")
             .merge(scibert, on=["article_id", "dataset_id"], how="inner")
)

# Filter out 'Missing'
df = df[df["pred_label"] != "Missing"].reset_index(drop=True)

# Load embedder
embedder = SentenceTransformer("/kaggle/input/all-minilm-l6-v2-and-deps/all-MiniLM-L6-v2/")
embed_window = embedder.encode(df["window"].tolist(), batch_size=64, show_progress_bar=True)
embed_dsid = embedder.encode(df["dataset_id"].tolist(), batch_size=64, show_progress_bar=True)

# One-hot types
llm_onehot = pd.get_dummies(df["type_llm"], prefix="llm")
scibert_onehot = pd.get_dummies(df["pred_label"], prefix="scibert")
llm_onehot = llm_onehot.reindex(columns=["llm_Primary", "llm_Secondary"], fill_value=0)
scibert_onehot = scibert_onehot.reindex(columns=["scibert_Primary", "scibert_Secondary"], fill_value=0)

# Features
X = np.hstack([
    embed_window,
    embed_dsid,
    df[["logit_primary", "logit_secondary"]].values,
    df[["logprob_A", "logprob_B"]].values,
    llm_onehot.values,
    scibert_onehot.values
])

# Normalize (same transform as training)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 🧠 Define model
class Classifier(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 2)
        )
    def forward(self, x):
        return self.net(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Classifier(input_dim=X.shape[1]).to(device)
model = torch.load(MODEL_PATH, weights_only=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Predict
with torch.no_grad():
    inputs = torch.tensor(X, dtype=torch.float32).to(device)
    logits = model(inputs)
    preds = torch.argmax(logits, dim=1).cpu().numpy()

# Map predictions
id2label = {0: "Primary", 1: "Secondary"}
df["type"] = [id2label[p] for p in preds]

# Create submission
submission = df[["article_id", "dataset_id", "type"]].copy()
submission.insert(0, "row_id", submission.index)
submission.to_csv(OUTPUT_PATH, index=False)

print(f"✅ Submission saved to {OUTPUT_PATH}")


Writing /tmp/src/nn_infer.py


In [30]:
%%writefile /tmp/src/Failures.py
import polars as pl

# Load predictions
pred_df = pl.read_csv("/kaggle/working/submission_nn.csv")

# Load official labels
truth_df = pl.read_csv("/kaggle/input/make-data-count-finding-data-references/train_labels.csv")

# Rename truth type for disambiguation
truth_df = truth_df.rename({"type": "type_truth"})

# Full outer join on article_id and dataset_id
joined = pred_df.join(truth_df, on=["article_id", "dataset_id"], how="right")
print(len(joined))
# Add error type column
error_expr = (
    pl.when(pl.col("type").is_null() & pl.col("type_truth").is_not_null()).then(pl.lit("false_negative"))
    .when(pl.col("type").is_not_null() & pl.col("type_truth").is_null()).then(pl.lit("false_positive"))
    .when(
        pl.col("type").is_not_null()
        & pl.col("type_truth").is_not_null()
        & (pl.col("type") != pl.col("type_truth"))
    ).then(pl.lit("type_mismatch"))
    .otherwise(pl.lit("correct"))
)

joined = joined.with_columns(error_expr.alias("error_type"))

# Extract error subsets
false_negatives = joined.filter(pl.col("error_type") == "false_negative")
false_positives = joined.filter(pl.col("error_type") == "false_positive")
type_mismatches = joined.filter(pl.col("error_type") == "type_mismatch")

# Report
print(f"False negatives: {false_negatives.shape[0]}")
print(f"False positives: {false_positives.shape[0]}")
print(f"Type mismatches: {type_mismatches.shape[0]}")

# Save detailed breakdowns
false_negatives.write_csv("/kaggle/working/false_negatives.csv")
false_positives.write_csv("/kaggle/working/false_positives.csv")
type_mismatches.write_csv("/kaggle/working/type_mismatches.csv")


Writing /tmp/src/Failures.py


In [31]:
! cat /tmp/logs/project.log

INFO 2025-07-23 15:22:18  [post_filter.py:52 - main()] all - f1: 0.6891 [481/196/238]

INFO 2025-07-23 15:22:18  [post_filter.py:52 - main()] doi - f1: 0.5889 [164/68/161]

INFO 2025-07-23 15:22:18  [post_filter.py:52 - main()] acc - f1: 0.7557 [317/128/77]

INFO 2025-07-23 15:22:18  [post_filter.py:53 - main()] all - f1: 0.5759 [402/275/317]

INFO 2025-07-23 15:22:18  [post_filter.py:53 - main()] doi - f1: 0.4596 [128/104/197]

INFO 2025-07-23 15:22:18  [post_filter.py:53 - main()] acc - f1: 0.6532 [274/171/120]

INFO 2025-07-30 10:24:30  [post_filter_scibert.py:56 - main()] all - f1: 0.5960 [486/426/233]

INFO 2025-07-30 10:24:30  [post_filter_scibert.py:56 - main()] doi - f1: 0.5795 [164/77/161]

INFO 2025-07-30 10:24:30  [post_filter_scibert.py:56 - main()] acc - f1: 0.6047 [322/349/72]

INFO 2025-07-30 10:24:30  [post_filter_scibert.py:57 - main()] all - f1: 0.5861 [478/434/241]

INFO 2025-07-30 10:24:30  [post_filter_scibert.py:57 - main()] doi - f1: 0.5583 [158/83/167]

INFO 2025-07-30 10:24:30  [post_filter_scibert.py:57 - main()] acc - f1: 0.6009 [320/351/74]
